## Dependencias

In [10]:
import numpy as np
import pandas as pd
from itertools import chain
import matplotlib.pyplot as plt
import seaborn as sns

## Población Inicial

In [11]:
def crear_individuo(num_genes=20):
    return "".join([str(int(x>0.5)) for x in np.random.uniform(size=num_genes)])

num_genes = 10
tam_pob = 100
pob = [crear_individuo(num_genes=num_genes) for _ in range(tam_pob)]

## Selección

In [12]:
def objetivo(x):
    return sum([int(g) for g in x])

def seleccion(objetivo,pob,k = 3):
    pob = pd.DataFrame({'x':pob}).sample(k)
    pob['aptitud'] = pob['x'].map(objetivo)
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1)['x'].values[0]

aptos = [seleccion(objetivo,pob) for _ in range(tam_pob)]

In [13]:
aptos[:2]

['1111011011', '0111101100']

## Reproducción

In [14]:
def combinacion(padre1,padre2,proba):
    #Los hijos son por defecto copias de los padres
    padre1 = [g for g in padre1]
    padre2 = [g for g in padre2]
    hijos = [padre1,padre2]
    if np.random.uniform()<proba:
        #Seleccionar un punto de combinación que no esté en los extremos
        i = np.random.choice(range(1,len(padre1)-1))
        hijos = padre1[:i]+padre2[i:],padre2[:i]+padre1[i:]
    return ["".join(h) for h in hijos]

## Mutación

In [15]:
def mutacion(individuo,proba=0.5):
    return "".join([str(1-int(g)) if r<proba else g  for g,r in zip(individuo,
                                                                    np.random.uniform(size=len(individuo)))])

In [16]:
hijos = [combinacion(*aptos[i:i+2],0.8) for i in range(0,100,2)]
hijos = list(map(lambda x: mutacion(x),chain(*hijos)))

## Algoritmo genético

In [17]:
def alg_genetico(objetivo,
                 n_genes=20,
                 n_generaciones=100,
                 tam_pob=100,
                 proba_comb=0.9,
                 proba_mut=1/20):
    pob = [crear_individuo(num_genes=n_genes) for _ in range(tam_pob)]
    for gen in range(n_generaciones):
        aptos = [seleccion(objetivo,pob) for _ in range(tam_pob)]
        hijos = [combinacion(*aptos[i:i+2],proba_comb) for i in range(0,tam_pob,2)]
        hijos = list(map(lambda x: mutacion(x,proba=proba_mut),chain(*hijos))) 
        pob = hijos[:]
    pob = pd.DataFrame({'x':pob})
    pob['aptitud'] = pob['x'].map(objetivo)
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1).values.tolist()[0]

In [18]:
alg_genetico(objetivo,n_generaciones=10)

['11111111111111111111', 20]

## Optimización continua $Z=\exp({-\frac{1}{3}x^{3}+x-y^{2}})$ 

In [19]:
espacio_de_busqueda = [[-2,2],[-2,2]]
n_genes = 32
tam_pob = 100
p_mutacion = 1/(n_genes*len(espacio_de_busqueda))

def Z(x,y):
    return np.exp(-1/3*x**3+x-y**2)

def decode(limites,n_genes,x):
    max_ = 2**(n_genes//2)
    x = [int(x[i*n_genes//2:n_genes//2*(i+1)],2) for i in range(len(limites))]
    x = [limites[i][0]+(v/max_)*(limites[i][1]-limites[i][0]) for i,v in enumerate(x)]
    return tuple(x)

def seleccion(objetivo,pob,k = 3,decodificar=False):
    if decodificar:
        decodificados = list(map(lambda x:decode(espacio_de_busqueda,n_genes,x),pob))
        pob = pd.DataFrame({'x':pob,'d':decodificados}).sample(k)
        pob['aptitud'] = pob['d'].map(lambda x:objetivo(*x))
    else:
        pob = pd.DataFrame({'x':pob}).sample(k)
        pob['aptitud'] = pob['x'].map(lambda x:objetivo(x))
    
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1)['x'].values[0]

pob = [crear_individuo(num_genes=n_genes) for _ in range(tam_pob)]

def alg_genetico(objetivo,
                 n_genes=20,
                 n_generaciones=100,
                 tam_pob=100,
                 proba_comb=0.9,
                 proba_mut=1/20,
                 tam_muestra = 3,
                 decodificar=True):
    pob = [crear_individuo(num_genes=n_genes) for _ in range(tam_pob)]
    
    for gen in range(n_generaciones):
        aptos = [seleccion(objetivo,pob,k=tam_muestra,decodificar=decodificar) for _ in range(tam_pob)]
        hijos = [combinacion(*aptos[i:i+2],proba_comb) for i in range(0,tam_pob,2)]
        hijos = list(map(lambda x: mutacion(x,proba=proba_mut),chain(*hijos))) 
        pob = hijos[:]
    if decodificar:
        decodificados = list(map(lambda x:decode(espacio_de_busqueda,n_genes,x),pob))
        pob = pd.DataFrame({'x':pob,'d':decodificados})
        pob['aptitud'] = pob['d'].map(lambda x:objetivo(*x))
    else:
        pob = pd.DataFrame({'x':pob})
        pob['aptitud'] = pob['x'].map(objetivo)
    pob.sort_values(by='aptitud',inplace=True)
    return pob.tail(1).values.tolist()[0]


print(alg_genetico(Z,
                   n_genes=n_genes,
                   tam_pob=tam_pob,
                   proba_mut=p_mutacion,
                   n_generaciones=100,
                   decodificar=True))

['10111111111111110111111111111111', (0.99993896484375, -6.103515625e-05), 1.947734026543074]
